### 將離散站點以反距離權重法計算出擴散圖

In [3]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.font_manager as fm
from matplotlib.cm import ScalarMappable

from matplotlib import colors
from descartes import PolygonPatch
from matplotlib.patches import PathPatch
from matplotlib.path import Path

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Microsoft YaHei', 'STHeiti']
plt.rcParams['axes.unicode_minus'] = False

In [4]:
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely import wkt
import math
import aqi_helper as helper

### 以已知網格點計算未知網格點值

In [5]:
def calc_idw_value(unknowncell, knowncells):
    sumwei = 0.0
    sumpm = 0.0
    wei = 0.0
    for index, row in knowncells.iterrows():
        px = unknowncell['x']
        py = unknowncell['y']
        pm = row['value']
        diss = math.sqrt(math.pow((px - row['lng']), 2) + math.pow((py - row['lat']), 2))
        dis = diss * 345
        if dis < 300:
            if dis < 1:
                dis = 1
            wei = 1 / math.pow(dis, 3)
            sumpm += pm * wei
            sumwei += wei
    if sumwei > 0:
        return round(sumpm / sumwei)
    else:
        return -1

### 生成台灣範圍的空網格後，將離散測站點的資料丟給上面的calc_idw_value()計算網格值

In [6]:
def calculate_idw(data): #計算網格IDW數值
    df = data #將測站site、lng、lat、value，DataFrame輸入

    # 將非數值轉為 NaN 並排除（這一步是關鍵）
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.dropna(subset=['value'])  # <--- 只用有數值的測站進行插值

    if df.empty:
        return np.full((120, 120), -1)  # 全部無資料時直接回傳 -1 網格
    
    df['value'].astype(float)#將value欄位轉成數值型態
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lng, df.lat))
    gdf.crs = 'EPSG:3826'
    gdf['geometry'] = gdf.geometry.buffer(0.015)

    iInterval = (122.7 - 119) / 120
    jInterval = (25.5 - 21.8) / 120
    result = []
    index1 = 0
    index2 = 0
    for j in np.arange(25.5, 21.8, -jInterval):
        nested = []
        for i in np.arange(119, 122.7, iInterval):
            point = {'x': i, 'y': j}
            if helper.is_in_area(index1, index2):
                nested.append(calc_idw_value(point, df))
            else:
                nested.append(-1)
            index2 += 1
        result.append(nested)
        index1 += 1
        index2 = 0

    result = np.array(result)

    return result

### 讀取台灣地圖gml檔與測站座標檔，接著進行IDW插值主函式

In [7]:
# --- 地圖與測站座標 ---
taiwan_map = gpd.read_file('./TOWN_MOI_1131028.gml')

site = pd.read_csv('./Preview_Data.csv')[['sitename','twd97lon','twd97lat']]
site.columns = ['site','lng','lat']
excluded_sites = ['富貴角', '馬祖', '金門', '馬公']
site = site[~site['site'].isin(excluded_sites)]

# --- 插值主函數 ---
def interpolate_by_idw(factor, input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for year in range(2015, 2020):
        input_path = os.path.join(input_dir, factor, f"{factor}_merged_{year}.csv")
        if not os.path.exists(input_path):
            print(f"❌ 檔案不存在: {input_path}")
            continue

        df = pd.read_csv(input_path)
        if '日期' not in df.columns:
            print(f"⚠️ 檔案格式錯誤（缺少日期）: {input_path}")
            continue

        df = df.set_index('日期')
        print(f"📂 處理 {factor} - {year}，共 {len(df)} 筆日期")

        m2 = []
        for count, date_str in enumerate(df.index):
            df_row = pd.DataFrame(df.iloc[count]).reset_index()
            df_row.columns = ['site', 'value']

            # 合併座標資料
            df_merge = pd.merge(site, df_row, on='site', how='inner')

            if df_merge['value'].isna().all():
                print(f"⚠️ {date_str} 無任何資料，跳過")
                m2.append(np.full((120, 120), np.nan))
                continue

            grid = calculate_idw(df_merge)
            m2.append(grid)
            print(f"{year} 年 {factor} 第 {count+1}/{len(df)} 筆完成")

        m2_array = np.round(np.array(m2), 2)

        # 儲存每筆（每週或每月）為獨立 CSV
        for index, grid in enumerate(m2_array, start=1):
            output_file = os.path.join(output_dir, f"{factor}_{year}_week_{index}.csv")
            pd.DataFrame(grid).to_csv(output_file, index=False)
            print(f"✅ 輸出：{output_file}")

# --- 執行程式 ---
if __name__ == "__main__":
    factors = ["NO", "NO2", "NOx", "O3", "PM10", "PM2.5", "SO2"]
    input_dir = './4_interpolated_yearly_stationwise'
    output_base = './5_grid_output'

    for factor in factors:
        factor_folder = os.path.join(output_base, factor.replace('.', ''))  # PM2.5 → PM25
        interpolate_by_idw(factor.replace('.', ''), input_dir, factor_folder)


📂 處理 NO - 2015，共 52 筆日期
2015 年 NO 第 1/52 筆完成
2015 年 NO 第 2/52 筆完成
2015 年 NO 第 3/52 筆完成
2015 年 NO 第 4/52 筆完成
2015 年 NO 第 5/52 筆完成
2015 年 NO 第 6/52 筆完成
2015 年 NO 第 7/52 筆完成
2015 年 NO 第 8/52 筆完成
2015 年 NO 第 9/52 筆完成
2015 年 NO 第 10/52 筆完成
2015 年 NO 第 11/52 筆完成
2015 年 NO 第 12/52 筆完成
2015 年 NO 第 13/52 筆完成
2015 年 NO 第 14/52 筆完成
2015 年 NO 第 15/52 筆完成
2015 年 NO 第 16/52 筆完成
2015 年 NO 第 17/52 筆完成
2015 年 NO 第 18/52 筆完成
2015 年 NO 第 19/52 筆完成
2015 年 NO 第 20/52 筆完成
2015 年 NO 第 21/52 筆完成
2015 年 NO 第 22/52 筆完成
2015 年 NO 第 23/52 筆完成
2015 年 NO 第 24/52 筆完成
2015 年 NO 第 25/52 筆完成
2015 年 NO 第 26/52 筆完成
2015 年 NO 第 27/52 筆完成
2015 年 NO 第 28/52 筆完成
2015 年 NO 第 29/52 筆完成
2015 年 NO 第 30/52 筆完成
2015 年 NO 第 31/52 筆完成
2015 年 NO 第 32/52 筆完成
2015 年 NO 第 33/52 筆完成
2015 年 NO 第 34/52 筆完成
2015 年 NO 第 35/52 筆完成
2015 年 NO 第 36/52 筆完成
2015 年 NO 第 37/52 筆完成
2015 年 NO 第 38/52 筆完成
2015 年 NO 第 39/52 筆完成
2015 年 NO 第 40/52 筆完成
2015 年 NO 第 41/52 筆完成
2015 年 NO 第 42/52 筆完成
2015 年 NO 第 43/52 筆完成
2015 年 NO 第 44/52 筆完成
2015 年 NO 第 45/52